In [1]:
import os
import numpy as np
import imgaug.augmenters as iaa
from PIL import Image

In [2]:

# Paths
input_dir = "../artifacts/rendered_images"  # Folder containing the generated images
augmented_dir = "../artifacts/augmented_outputs"  # Folder to save augmented images

# Ensure output directory exists
os.makedirs(augmented_dir, exist_ok=True)

# Define augmentation pipeline
augmentation_pipeline = iaa.Sequential([
    iaa.Sometimes(0.5, iaa.PerspectiveTransform(scale=(0.01, 0.15))),  # Writing angle variations
    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 1.5))),  # Slight blur
    iaa.Sometimes(0.3, iaa.AdditiveGaussianNoise(scale=(10, 30))),  # Noise for realism
    iaa.Sometimes(0.5, iaa.LinearContrast((0.75, 1.25))),  # Contrast variations
])

def add_coffee_stain(image_np):
    """Simulates random coffee stains or smudges."""
    stain_augmenter = iaa.Sometimes(
        0.4,  # 40% probability of adding a stain
        iaa.BlendAlphaSimplexNoise(iaa.Multiply((0.7, 1.3)), per_channel=True)
    )
    return stain_augmenter.augment_image(image_np)

def vary_pen_pencil_thickness(image_np):
    """Varies pen thickness and color."""
    stroke_augmenter = iaa.OneOf([
        iaa.Sometimes(0.5, iaa.AddToHueAndSaturation((-30, 30))),  # Color tone variation
        iaa.Sometimes(0.7, iaa.Multiply((0.8, 1.2))),  # Ink intensity variation
    ])
    return stroke_augmenter.augment_image(image_np)

def augment_image(image_path, output_dir, augmentations, num_augmentations=5):
    """Applies augmentations to an image and saves multiple augmented versions."""
    image = Image.open(image_path).convert("RGB")
    image_np = np.array(image)  # Convert to NumPy array

    for i in range(num_augmentations):
        augmented_image_np = augmentations.augment_image(image_np)
        augmented_image_np = add_coffee_stain(augmented_image_np)
        augmented_image_np = vary_pen_pencil_thickness(augmented_image_np)

        augmented_image = Image.fromarray(augmented_image_np)  # Convert back to PIL Image
        output_path = os.path.join(output_dir, f"{os.path.basename(image_path).split('.')[0]}_aug_{i}.png")
        augmented_image.save(output_path)

# Override NumPy bool usage in imgaug (Fix for future runs)
np.bool = np.bool_  # This prevents imgaug from using the deprecated alias

# Process all images in the input directory
for image_file in os.listdir(input_dir):
    if image_file.lower().endswith((".png", ".jpg", ".jpeg")):
        image_path = os.path.join(input_dir, image_file)
        augment_image(image_path, augmented_dir, augmentation_pipeline)

print("Augmentation complete. Augmented images saved in:", augmented_dir)


Augmentation complete. Augmented images saved in: ../artifacts/augmented_outputs
